**<font color=blue size=5 face=雅黑>千千音樂</font>**<br>
https://play.taihe.com/<br>
**<font color=blue size=3 face=雅黑>Introduction</font>**<br>
獲取歌曲下載網址：https://zhuanlan.zhihu.com/p/91327595<br>
https://mrasong.com/a/baidu-mp3-api<br>
https://www.jianshu.com/p/e3cd7d43d02f

In [1]:
import re
import os
import json
import requests
import pandas as pd
import numpy as np
import urllib.parse
from bs4 import BeautifulSoup

In [51]:
# URL Encoding query : https://www.urlencoder.io/python/
# 簡體字和繁體字會不一樣
def get_EncodeURL(query):
    queryURL = urllib.parse.quote(query)
    print(query + ' → ' + queryURL)
    return queryURL

get_EncodeURL('欧美')

欧美 → %E6%AC%A7%E7%BE%8E


'%E6%AC%A7%E7%BE%8E'

In [52]:
# 獲取音樂的songid (法一)
def get_songid_1(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    # 選擇 class='song-title'中所有<span>標籤
    title = soup.findAll("span", {"class": "song-title"})
    #print(title)

    df_song_1 = pd.DataFrame()
    list_song_name = []
    list_song_id = []

    for i in range(len(title)):
        x = title[i]
        song_name = x.text
        song_href = x.a.get('href')
        song_id = song_href.split('/')[2]
        list_song_name.append(song_name)
        list_song_id.append(song_id)
    
    df_song_1['SongName'] = pd.Series(list_song_name)
    df_song_1['SongID'] = pd.Series(list_song_id)
    #df_song1.to_csv('BaiduMusic_csv/df_song_1.csv')
    return df_song_1

In [45]:
get_songid_1('https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E?start=0&size=20&third_type=0')

,SongName,SongID
0,Need To Know,604077590
1,I Want It That Way,297259321
2,I Knew You Were Trouble,335898602
3,Demons,289627704
4,Panama,323997901
5,è°è®°å¾,251705016
6,Spectre,549264975
7,Happy Together,317646298
8,Thank You,315086661
9,You Raise Me Up,327900966


In [4]:
# 獲取音樂的songid (法二)
def get_songid_2(url):
    response = requests.get(url=url)
    html = response.text
    # \d找到數字，輸出的型態是list
    sids = re.findall(r'href="/song/(\d+)"', html)
    del sids[0]
    df_song_2 = pd.DataFrame()
    #print(len(sids))
    df_song_2['SongID'] = pd.Series(sids)
    return df_song_2

In [ ]:
get_songid_2('https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E?start=0&size=20&third_type=0')

In [6]:
# 判斷pandas的兩個DataFrame是否相同
url = 'https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E?start=0&size=20&third_type=0'
df1 = get_songid_1(url)
df1 = df1[['SongID']]
df2 = get_songid_2(url)
results = df1.equals(df2)
results

True

In [7]:
#獲取下載連結
def get_music_url(songid):
    api_url = f'http://musicapi.taihe.com/v1/restserver/ting?method=baidu.ting.song.playAAC&format=jsonp&songid={songid}&from=web'
    response = requests.get(api_url.format(songid=songid))
    data = response.json()
    #print(data)
    try:
        artist_name = data['songinfo']['artist']
        music_name = data['songinfo']['title']
        music_url = data['bitrate']['file_link']
        return artist_name, music_name, music_url
    except Exception as e:
        print(e)

In [8]:
get_music_url(328872354)

('Coldplay',
 'Easy To Please',
 'http://audio04.dmhmusic.com/71_53_T10016239086_128_4_1_0_sdk-cpm/cn/0101/M00/46/C5/ChR45VmKA86AIlHzACxwf7wb6BI876.mp3?xcode=15cb3ab040e95d6e5b0b895793f2bab80ee5f1a')

In [9]:
# 儲存音樂
def save_mp3_file(filename, content):
    filepath = 'BaiduMusic_mp3/'+ filename
    with open(filepath, mode = "wb") as f:
        f.write(content)
    print(filepath)

In [10]:
# 下載音樂.mp3
def download_music(artist_name, music_name, music_url):
    response = requests.get(music_url)
    content = response.content
    # mp3檔案命名規則
    save_mp3_file(artist_name + '_' + music_name + '.mp3', content)

In [11]:
# 取得動態歌詞.lrc
def get_lrc(songid):
    api_url = f'http://musicapi.taihe.com/v1/restserver/ting?method=baidu.ting.song.lry&songid={songid}&from=web'
    response = requests.get(api_url.format(songid = songid))
    data = response.json()
    return data

In [12]:
# 下載動態歌詞
# 解決UnicodeDecodeError: 'cp950'問題
def save_lrc_file(data, artist_name, music_name):
    content = data['lrcContent'].replace('\r','')
    filepath = 'BaiduMusic_lrc/'+ artist_name + '_' + music_name +'.lrc'
    with open(filepath, 'w', encoding="utf-8") as file:
        file.write(content)
    print(filepath)

In [13]:
# 檔案名稱所有非法字元都轉義：https://www.itread01.com/content/1549125182.html
def validateTitle(title):
    rstr = r"[\/\\\:\*\?\"\<\>\|]" # '/ \ : * ? " < > |'
    new_title = re.sub(rstr, " ", title)  # 替換為空白
    return new_title

In [16]:
# 下一頁
# %E6%AC%A7%E7%BE%8E 欧美
url = 'https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E?start=0&size=20&third_type=0'
list_next_page = []

def get_nextpage(url):
    page1 = requests.get(url)
    soup1 = BeautifulSoup(page1.text, "html.parser")
    results = soup1.find("a", {"class": "page-navigator-next"}).attrs["href"]
    page_href = results.replace("\t", "").replace(" ", "").replace("\n", "") 
    # 下一頁網址
    next_page_url = 'https://music.taihe.com' + page_href
    print(next_page_url)
    url = next_page_url
    try:
        list_next_page.append(url)
        get_nextpage(url=next_page_url)
    except:
        pass
    
    return list_next_page

In [15]:
list_next_page = get_nextpage(url)
list_next_page.insert(0, url)
print(list_next_page)

NameError: name 'get_nextpage' is not defined

In [17]:
# 下一頁轉成DataFrame並存成csv
df_next_page = pd.DataFrame()
df_next_page['Next_Page'] = pd.Series(list_next_page)
df_next_page.to_csv('BaiduMusic_csv/df_next_page.csv', index=False)

In [19]:
# 匯入下一頁網址的csv轉成DataFrame
read_df_next_page = pd.read_csv('BaiduMusic_csv/df_next_page.csv', delimiter = ',')
read_df_next_page

,Next_Page
0,https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E...
1,https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E...
2,https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E...
3,https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E...
4,https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E...
5,https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E...
6,https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E...
7,https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E...
8,https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E...
9,https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E...


In [20]:
# df_next_page_list的DataFrame匯入轉成list
read_df_next_page_list = read_df_next_page.values.tolist()
read_df_next_page_list

[['https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E?start=0&size=20&third_type=0'],
 ['https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E?start=20&size=20&third_type=0'],
 ['https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E?start=40&size=20&third_type=0'],
 ['https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E?start=60&size=20&third_type=0'],
 ['https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E?start=80&size=20&third_type=0'],
 ['https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E?start=100&size=20&third_type=0'],
 ['https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E?start=120&size=20&third_type=0'],
 ['https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E?start=140&size=20&third_type=0'],
 ['https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E?start=160&size=20&third_type=0'],
 ['https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E?start=180&size=20&third_type=0'],
 ['https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E?start=200&size=20&third_type=0'],
 ['https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E?start=220&size=20&third_type=0']

In [36]:
for w in range(41,50):
#for w in range(len(list_next_page)):
    #url = read_df_next_page_list[w][0]
    url = 'https://music.taihe.com/songlist/566348195'
    print('='*80 + '\n' + 'Page' + str(w+1) + ' ' +'[ URL ]')
    print(url + '\n' + '='*80)
    df_song_1 = get_songid_1(url)
    #df_song_1 = get_songid_1(list_next_page[w])
    for v in range(len(df_song_1)):
        songid = df_song_1['SongID'][v]
        print('SongID :',songid)
        try:
            artist_name, music_name, music_url = get_music_url(songid)
            artist_name = validateTitle(artist_name)
            music_name = validateTitle(music_name)
            download_music(artist_name, music_name, music_url)
            data = get_lrc(songid)
            save_lrc_file(data, artist_name, music_name)
        except:
            pass
        print('-'*80)

Page42 [ URL ]
https://music.taihe.com/tag/%E6%AC%A7%E7%BE%8E?start=820&size=20&third_type=0
SongID : 330450645
mp3/Mickael Carreira_Dança Comigo (feat. My-Kul Leeric).mp3
--------------------------------------------------------------------------------
SongID : 325012845
mp3/Laurel_To the Hills.mp3
--------------------------------------------------------------------------------
SongID : 318382717
mp3/Denison Witmer_Are You A Dreamer .mp3
--------------------------------------------------------------------------------
SongID : 317803277
mp3/Say Lou Lou_Everything We Touch.mp3
--------------------------------------------------------------------------------
SongID : 314090851
mp3/Tom Glazer_The Arms of Abraham.mp3
--------------------------------------------------------------------------------
SongID : 313133621
mp3/Pretty & Nice_Q_Q.mp3
--------------------------------------------------------------------------------
SongID : 312519820
mp3/Adventure Club_Wonder (feat. The Kite String Tang

mp3/Bishop Allen_Click, Click, Click, Click.mp3
--------------------------------------------------------------------------------
SongID : 295803476
mp3/Bear Hands_High Society.mp3
--------------------------------------------------------------------------------
SongID : 293758877
mp3/Robert Randolph & The Family Band_Ain't Nothing Wrong With That.mp3
--------------------------------------------------------------------------------
SongID : 292718408
mp3/Various Artists_Take On Me (Karaoke Version).mp3
--------------------------------------------------------------------------------
SongID : 291878462
mp3/Dave Chamberlain,Matt Home,Stacey Kent_Too Darn Hot.mp3
--------------------------------------------------------------------------------
SongID : 282118206
mp3/B.J. Thomas_Raindrops Keep Fallin' on My Head.mp3
--------------------------------------------------------------------------------
SongID : 279733025
mp3/The Dickies_Banana Splits (The Tra La La Song).mp3
--------------------------

mp3/Attraction_I Count Each Minute.mp3
--------------------------------------------------------------------------------
SongID : 332918247
mp3/The Platters_Smoke Gets in Your Eyes.mp3
--------------------------------------------------------------------------------
SongID : 331451802
mp3/Vanessa Williams_Save the Best for Last.mp3
--------------------------------------------------------------------------------
SongID : 330471730
mp3/Anton Karas_The Harry Lime Theme.mp3
--------------------------------------------------------------------------------
SongID : 330122101
mp3/Chris Isaak_Wicked Games.mp3
--------------------------------------------------------------------------------
SongID : 329877485
mp3/The Carpenters_Sing.mp3
--------------------------------------------------------------------------------
SongID : 328229863
mp3/Adventure Club,Dallask_Crash 2.0 (Adventure Club vs Dallask).mp3
--------------------------------------------------------------------------------
SongID : 3281283

SongID : 279394118
mp3/Berlin_Take My Breath Away.mp3
--------------------------------------------------------------------------------
SongID : 278345287
mp3/Mt. Wolf_Cry Wolf.mp3
--------------------------------------------------------------------------------
SongID : 277902969
mp3/Open Mike Eagle,Paul White_The Curse of Hypervigilance (In Politics, Romance, and Cohabitation).mp3
--------------------------------------------------------------------------------
SongID : 552636423
mp3/Del Barber_Everything Is Not Enough.mp3
--------------------------------------------------------------------------------
SongID : 550305523
mp3/John Smith_Rippin' Tickets at the Tilt-A-Whirl.mp3
--------------------------------------------------------------------------------
SongID : 549811664
mp3/Le Vent Du Nord_Vigneron.mp3
--------------------------------------------------------------------------------
SongID : 549584394
mp3/mikedecline_Chicago.mp3
--------------------------------------------------------

In [34]:
# 測試用 https://music.taihe.com/song/297200802
songid = '297290605'
artist_name, music_name, music_url = get_music_url(songid)
# 檔名去除無效字元
artist_name = validateTitle(artist_name)
print(artist_name)
music_name = validateTitle(music_name)
print(music_name)
try:
    download_music(artist_name, music_name, music_url)
    data = get_lrc(songid)
    save_lrc_file(data, artist_name, music_name)
except:
    pass

Laura Welsh
Break the Fall
mp3/Laura Welsh_Break the Fall.mp3


In [53]:
# 透過關鍵字獲取歌曲列表
def keywords_getlist(query_URL):
    api_url = f'http://mp3.baidu.com/dev/api/?tn=getinfo&ct=0&word={query_URL}ie=utf-8&format=json'
    response = requests.get(api_url.format(query_URL=query_URL))
    data = response.json()
    print(data)

In [55]:
query_URL = get_EncodeURL('Katy Perry')
keywords_getlist(query_URL)

Katy Perry → Katy%20Perry


JSONDecodeError: Expecting value: line 2 column 1 (char 47)

In [46]:
# 獲取歌手歌曲列表
# Katy Perry (ID=2716)
def get_artist_songlist(astistid, num):
    api_url = f'http://musicapi.taihe.com/v1/restserver/ting?method=baidu.ting.artist.getSongList&tinguid={astistid}&limits={num}&use_cluster=1&order=2'
    response = requests.get(api_url.format(songid=songid, num=num))
    data = response.json()
    print(data)

In [57]:
get_artist_songlist(1512, 5)

{'songlist': [{'artist_id': '1315', 'all_artist_ting_uid': '1512', 'all_artist_id': '1315', 'language': '国语', 'publishtime': '1995-01-01', 'album_no': '6', 'versions': '', 'pic_big': 'http://qukufile2.qianqian.com/data2/pic/f8d0cb5e990299313455651c11fb9546/268334969/268334969.jpg@s_2,w_150,h_150', 'pic_small': 'http://qukufile2.qianqian.com/data2/pic/f8d0cb5e990299313455651c11fb9546/268334969/268334969.jpg@s_2,w_90,h_90', 'country': '港台', 'area': '1', 'lrclink': '', 'hot': '0', 'file_duration': '231', 'del_status': '0', 'resource_type': '0', 'resource_type_ext': '0', 'copy_type': '0', 'relate_status': '0', 'all_rate': '320,128', 'has_mv_mobile': 0, 'toneid': '0', 'bitrate_fee': '{"0":"129|-1","1":"-1|-1"}', 'biaoshi': 'vip,perm-0', 'info': '', 'has_filmtv': '0', 'si_proxycompany': '深圳腾讯计算机系统有限公司福茂', 'song_id': '268620501', 'title': 'SOS Party', 'ting_uid': '1512', 'author': 'ASOS', 'album_id': '159399', 'album_title': '姐妹情深', 'is_first_publish': 0, 'havehigh': 2, 'charge': 0, 'has_mv':

In [64]:
# 獲取歌手資訊
def get_artist_info(astistid):
    api_url = f'http://musicapi.taihe.com/v1/restserver/ting?baidu.ting.artist.getInfo&tinguid={astistid}'
    response = requests.get(api_url.format(astistid=astistid))
    soup = BeautifulSoup(response.text, "html.parser")
    print(soup)

In [66]:
get_artist_info(1512)